## Exploring 'Movies' table
### 1 Library and duckdb file import

In [1]:
#initial exploration_movies.py

import duckdb, pandas as pd
from pathlib import Path

# create or connect if it already exists:
con = duckdb.connect("movielensfull33M.duckdb")

### 2 General dataset description

In [2]:
con.sql("DESCRIBE movies").df()


,column_name,column_type,null,key,default,extra
0,movieId,BIGINT,YES,None,None,None
1,title,VARCHAR,YES,None,None,None
2,genres,VARCHAR,YES,None,None,None


Comments:

- `movieID`: BIGINT (64-bit integer value)  
- `title`: movie name (VARCHAR, text)  
- `genres`: genre names (VARCHAR, text)

- `null`: indicates whether the column can contain null (NULL) values.  
  - In this case, it can.

- `key`: indicates whether the column is a primary key (PRIMARY KEY).  
  - It is not.

- `default`: shows the default value (DEFAULT).  
  - None.

- `extra`: displays additional information such as auto_increment or generated.  
  - None in this case.

In [3]:
con.sql("PRAGMA table_info('movies')").df()


,cid,name,type,notnull,dflt_value,pk
0,0,movieId,BIGINT,False,None,False
1,1,title,VARCHAR,False,None,False
2,2,genres,VARCHAR,False,None,False


Comments:
- `movieId`: BIGINT (64-bit integer value)  
- `title`: movie name (VARCHAR, text)  
- `genres`: genre names (VARCHAR, text)

- The column `notnull` is `False` for all fields.  
  - This means the table allows null (NULL) values.  
  - Since the table was created using `read_csv_auto`, DuckDB does not enforce NOT NULL constraints by default.

- The column `dflt_value` is `None` for all fields.  
  - This means there are no default values assigned when inserting data.

- The column `pk` is `False` for all fields.  
  - This indicates that no column is part of the table’s primary key.


### 3 Dataset individual basic exploration
#### 3.1 Dataset composition

In [4]:
# See first 10 rows
con.sql("SELECT * FROM movies LIMIT 10").df()


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


#### 3.2 Row counts

In [5]:
#Count total number of rows
con.sql("SELECT COUNT(*) AS total_movies FROM movies").df()

,total_movies
0,86537


Comments:

- This table is used to identify the movies present in the current database.  
- It lists the ID of each movie and its associated genre(s).  
- There are no primary keys.  
- The table contains a total of 86 537 movies.  
- Each movie can belong to more than one genre.


#### 3.3 Missing values

In [6]:
# Count number of missing values
con.sql("""
SELECT
    COUNT(*) - COUNT(movieId) AS missing_movieId,
    COUNT(*) - COUNT(title)   AS missing_title,
    COUNT(*) - COUNT(genres)  AS missing_genres
FROM movies
""").df()


,missing_movieId,missing_title,missing_genres
0,0,0,0


Comments:

 - There are no missing values


#### 3.4 Genres distribution

In [7]:
# See value distribution
con.sql("""
SELECT genres, COUNT(*) AS n
FROM movies
GROUP BY genres
ORDER BY n DESC
LIMIT 20
""").df()


,genres,n
0,Drama,12246
1,Documentary,8064
2,Comedy,7689
3,(no genres listed),7060
4,Comedy|Drama,3196
5,Drama|Romance,2788
6,Horror,2467
7,Comedy|Romance,2176
8,Thriller,1397
9,Comedy|Drama|Romance,1310


Comments

It is observed that there may be mixed genres, which can make it difficult to quantify each genre.  
For example: `Drama` , `Comedy` or `Drama|Comedy`, etc., which can complicate counting and analysis.

#### 3.5 Genre distribution with separations rows per genre

In [8]:
# Analyze genre distribution with separations rows per genre
con.sql("""
SELECT
    genre,
    COUNT(*) AS total_movies_with_genre,
    ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM movies), 2) AS percentage_of_all_movies
FROM (
    SELECT DISTINCT movieId, unnest(string_split(genres, '|')) AS genre
    FROM movies
)
GROUP BY genre
ORDER BY total_movies_with_genre DESC;
""").df()

,genre,total_movies_with_genre,percentage_of_all_movies
0,Drama,33681,38.92
1,Comedy,22830,26.38
2,Thriller,11675,13.49
3,Romance,10172,11.75
4,Action,9563,11.05
5,Documentary,9283,10.73
6,Horror,8570,9.90
7,(no genres listed),7060,8.16
8,Crime,6917,7.99
9,Adventure,5349,6.18


#### 3.6 Number of average movie genres

In [9]:
#See how many genres on average each movie has
con.sql("""
SELECT
    AVG(array_length(string_split(genres, '|'))) AS media_generos_por_filme
FROM movies
""").df()


,media_generos_por_filme
0,1.759802


#### 3.7 Top of movies with higher number of movie genre

In [10]:
#See movies with the highest number of genres
con.sql("""
SELECT
    title,
    genres,
    array_length(string_split(genres, '|')) AS n_generos
FROM movies
ORDER BY n_generos DESC, title
LIMIT 10
""").df()


,title,genres,n_generos
0,Rubber (2010),Action|Adventure|Comedy|Crime|Drama|Film-Noir|...,10
1,Motorama (1991),Adventure|Comedy|Crime|Drama|Fantasy|Mystery|S...,8
2,"Wonderful World of the Brothers Grimm, The (1962)",Adventure|Animation|Children|Comedy|Drama|Fant...,8
3,2012: An Awakening (2009),Action|Documentary|Drama|Horror|Mystery|Sci-Fi...,7
4,Aelita: The Queen of Mars (Aelita) (1924),Action|Adventure|Drama|Fantasy|Romance|Sci-Fi|...,7
5,Aqua Teen Hunger Force Colon Movie Film for Th...,Action|Adventure|Animation|Comedy|Fantasy|Myst...,7
6,Black Butler (2014),Action|Adventure|Crime|Drama|Fantasy|Horror|My...,7
7,Calcutta (1947),Action|Crime|Drama|Film-Noir|Mystery|Romance|T...,7
8,Creators: The Past (2020),Action|Adventure|Drama|Fantasy|Mystery|Romance...,7
9,Enchanted (2007),Adventure|Animation|Children|Comedy|Fantasy|Mu...,7


Comments:
- 39% of the movies can be identified as "Drama" (around 33K out of around 87K entries), followed by "Comedy" with around 26% of the movies (aound 23K entries)
- Even though there are no missing values, there are 7060 movies without a genre allocated ("no genres listed") - this is equivalent to 8.1% of the full dataset
- On average, a movie has 1.8 categories, having between 1 to 10 categories (assuming "no genres listed" is a category)
- Fun fact: the trailer of the movie that has the most categories ("Rubber") is... peculiar!

#### Close connection

In [11]:
con.close()
print("Connection closed.")

Connection closed.
